In [2]:
import numpy as np 
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

C:\Users\chimi\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\chimi\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
avocadoprice_df = pd.read_csv('C:/Users/chimi/Desktop/Tableau Projects/avocado/avocado cleaned.csv')

In [4]:
avocadoprice_df.head(5)

,Date,AveragePrice,TotalVolume,plu4046,plu4225,plu4770,TotalBags,SmallBags,LargeBags,XLargeBags,type,year,region
0,1/4/2015,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,1/4/2015,1.79,1373.95,57.42,153.88,0.00,1162.65,1162.65,0.00,0.0,organic,2015,Albany
2,1/11/2015,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
3,1/11/2015,1.77,1182.56,39.00,305.12,0.00,838.44,838.44,0.00,0.0,organic,2015,Albany
4,1/18/2015,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany


In [6]:

def get_lat_lon(region):
    try:
        geolocator = Nominatim(user_agent="AvocadoPriceDashboard/1.2", timeout=10)  # Increase timeout to 10 seconds
        location = geolocator.geocode(region + ", USA")  
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        print(f"Geocoding request for {region} timed out.")
        return None, None

# List of regions from your dataset
regions_from_dataset = avocadoprice_df['region'].unique()

# Dictionary to store latitude and longitude for each region
lat_lon_dict = {}

# Fetch latitude and longitude for each region
for region in regions_from_dataset:
    latitude, longitude = get_lat_lon(region)
    lat_lon_dict[region] = (latitude, longitude)

# Print the results
for region, (latitude, longitude) in lat_lon_dict.items():
    print(f"{region}: Latitude {latitude}, Longitude {longitude}")


Albany: Latitude 42.6511674, Longitude -73.754968
Atlanta: Latitude 33.7489924, Longitude -84.3902644
BaltimoreWashington: Latitude None, Longitude None
BirminghamMontgomery: Latitude None, Longitude None
Boise: Latitude 43.6166163, Longitude -116.200886
Boston: Latitude 42.3554334, Longitude -71.060511
BuffaloRochester: Latitude None, Longitude None
California: Latitude 36.7014631, Longitude -118.755997
Charlotte: Latitude 35.2272086, Longitude -80.8430827
Chicago: Latitude 41.8755616, Longitude -87.6244212
CincinnatiDayton: Latitude None, Longitude None
Columbus: Latitude 39.9622601, Longitude -83.0007065
DallasFtWorth: Latitude None, Longitude None
Denver: Latitude 39.7392364, Longitude -104.984862
Detroit: Latitude 42.3315509, Longitude -83.0466403
GrandRapids: Latitude None, Longitude None
GreatLakes: Latitude None, Longitude None
HarrisburgScranton: Latitude None, Longitude None
HartfordSpringfield: Latitude None, Longitude None
Houston: Latitude 29.7589382, Longitude -95.3676974

In [9]:
# Convert lat_lon_dict to a list of dictionaries suitable for DataFrame creation
data = [{'region': region, 'latitude': lat, 'longitude': lon} for region, (lat, lon) in lat_lon_dict.items()]

# Create a new DataFrame from the list of dictionaries
geo_df = pd.DataFrame(data)

In [10]:
geo_df

,region,latitude,longitude
0,Albany,42.651167,-73.754968
1,Atlanta,33.748992,-84.390264
2,BaltimoreWashington,NaN,NaN
3,BirminghamMontgomery,NaN,NaN
4,Boise,43.616616,-116.200886
...,...,...,...
56,Toledo,41.652914,-83.537817
57,TotalUS,NaN,NaN
58,West,31.803481,-97.093347
59,WestTexNewMexico,NaN,NaN


In [12]:
# Perform Linear Interpolation to fill in the missing values for longitude and latitude
new_df['latitude'] = new_df['latitude'].interpolate(method='linear')
new_df['longitude'] = new_df['longitude'].interpolate(method='linear')


In [13]:
new_df.head(5)

,region,latitude,longitude
0,Albany,42.651167,-73.754968
1,Atlanta,33.748992,-84.390264
2,BaltimoreWashington,37.038200,-94.993805
3,BirminghamMontgomery,40.327408,-105.597345
4,Boise,43.616616,-116.200886


In [16]:
new_df.isnull().sum()

region       0
latitude     0
longitude    0
dtype: int64

In [17]:
new_df.to_csv('citiesgeo.csv',index=False)